In [1]:
print('aga')
!echo $CONDA_DEFAULT_ENV


aga
lgd2


In [2]:
import lgd  # noqa, register custom modules
import argparse
import datetime
import os
import logging
import torch
from torch_geometric.graphgym.cmd_args import parse_args
from torch_geometric.graphgym.config import (cfg, dump_cfg,
                                             set_cfg, load_cfg,
                                             makedirs_rm_exist)
from torch_geometric.graphgym.loader import create_loader
from torch_geometric.graphgym.logger import set_printing
from torch_geometric.graphgym.optim import create_optimizer, \
    create_scheduler, OptimizerConfig
from torch_geometric.graphgym.model_builder import create_model
from torch_geometric.graphgym.train import GraphGymDataModule, train
from torch_geometric.graphgym.utils.comp_budget import params_count
from torch_geometric.graphgym.utils.device import auto_select_device
from torch_geometric.graphgym.register import train_dict, network_dict, register_network, act_dict
from torch_geometric import seed_everything
from lgd.asset.logger import create_logger
from lgd.loader.master_loader import load_dataset_master
from lgd.optimizer.extra_optimizers import ExtendedSchedulerConfig
from lgd.agg_runs import agg_runs
from lgd.finetuning import load_pretrained_model_cfg, \
    init_model_from_pretrained
from lgd.ddpm.LGD import DDPM, LatentDiffusion
from lgd.ddpm.LGD_Inductive import LatentDiffusionInductive
from lgd.train.pretrain_encoder import *
from lgd.encoder.atom_bond_encoder import *
from lgd.model.GraphTransformerEncoder import GraphTransformerEncoder # HERE 
from lgd.train.train_diffusion import *
from utils import print_gpu_usage
import sys
import torch.nn.functional as F

In [3]:

sys.argv = ["train_diffusion.py", "--cfg", "cfg/ogbg-molhiv-diffusion.yaml", "--repeat", '1']
args = parse_args()
# Load config file
set_cfg(cfg)
cfg.set_new_allowed(True)
load_cfg(cfg, args)
cfg.accelerator = 'cpu' #-eren
cfg.devices = None
loaders = create_loader()
train_loader, val_loader, test_loader = loaders


format OGB
here ogbg-molhiv
if ten once ./datasets/ogbg_molhiv/processed.pt
if ici


/home/eren/LatentGraphDiffusion/lgd/loader/master_loader.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(processed_path, map_location="cpu")
/home

In [4]:
model = eval(cfg.model.get('type', 'LatentDiffusion'))\
            (timesteps=cfg.diffusion.get('timesteps', 1000), conditioning_key=cfg.diffusion.conditioning_key,
             hid_dim=cfg.diffusion.hid_dim, parameterization=cfg.diffusion.get("parameterization", "x0"),
             cond_stage_key=cfg.diffusion.cond_stage_key, first_stage_config=cfg.diffusion.first_stage_config,
             cond_stage_config=cfg.diffusion.cond_stage_config, edge_factor=cfg.diffusion.get("edge_factor", 1.0),
             graph_factor=cfg.diffusion.get("graph_factor", 1.0),
             train_mode=cfg.diffusion.get("train_mode", 'sample')).to(torch.device(cfg.accelerator))

pretrained_dir = 'results/ogbg-molhiv-diffusion-ogbg-molhiv-orig-2025-03-02_15-14-20/0/ckpt/49.ckpt'
model = init_model_from_pretrained(
                model, pretrained_dir, cfg.pretrained.freeze_main,
                cfg.pretrained.reset_prediction_head
            )

LatentDiffusion: Running in x0-prediction mode
DiffusionWrapper has 0.60 M params.
Using first stage also as cond stage.


/home/eren/miniconda3/envs/lgd2/lib/python3.9/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
/home/eren/LatentGraphDiffusion/lgd/ddpm/LGD.py:555: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you 

In [5]:

def accuracy(model, loader):
    correct = 0.0
    total = 0.0
    for i, batch in enumerate(loader):
        
        graph_label = batch.y.clone().detach().cpu()
        graph_label = graph_label.squeeze(-1)
        batch_z = model.encode_first_stage(batch)
        node_decode, edge_decode, graph_decode = model.decode_first_stage(batch_z)
        graph_decode = graph_decode.squeeze(-1)
        probabilities = torch.sigmoid(graph_decode).cpu()
        hard_pred = (probabilities >= 0.5).int()
        assert(graph_decode.shape[0] == graph_label.shape[0])
        correct += torch.sum(hard_pred == graph_label)
        
        total += graph_decode.shape[0]
    
    print(correct/total)
    return correct/total


In [7]:
model.eval()
time_start = time.time()
generated_graphs = []
iter = 0
num_test_graphs = 0
for batch in test_loader:
    batch.split = 'test'
    batch.to(torch.device(cfg.accelerator))
    
    node_label, edge_label, graph_label = batch.x.clone().detach().flatten(), batch.edge_attr.clone().detach().flatten(), batch.y
    # the embed of labels and prefix are done in fine-tuning of the encoder, not pretraining
    batch.x_masked = batch.x.clone().detach()
    batch.edge_attr_masked = batch.edge_attr.clone().detach()
    ddim_steps = cfg.diffusion.get('ddim_steps', None)
    ddim_eta = cfg.diffusion.get('ddim_eta', 0.0)
    use_ddpm_steps = cfg.diffusion.get('use_ddpm_steps', False)

    _, node_decode, edge_decode, graph_decode, generated = model.inference(batch, ddim_steps=ddim_steps, ddim_eta=ddim_eta, use_ddpm_steps=use_ddpm_steps, visualize=False, return_all = True)
    for each in generated:
        generated_graphs.append(each)

    break

Sampling t: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [32:03<00:00,  1.92s/it]


In [9]:
import pickle
with open('generated_tuple.pkl', 'wb') as f:
    pickle.dump((node_decode, edge_decode, graph_decode, generated), f)

In [28]:
with open('generated_tuple.pkl', 'rb') as f:
    loaded_data = pickle.load(f)  
    node_decode, edge_decode, graph_decode, generated = loaded_data



for batch in test_loader:
    #BURA ONEMLI
    node_label = batch.x.clone().detach()
    edge_label = batch.edge_attr.clone().detach()
    if not cfg.train.pretrain.atom_bond_only:
        node_label = node_label[:, 0].flatten()
        edge_label = edge_label[:, 0].flatten()

    break
print(node_label.shape)
print(edge_label.shape)
gen_node, gen_edge, gen_graph = generated[0]
print(gen_node.shape)
print(gen_edge.shape)

torch.Size([3253])
torch.Size([102789])
torch.Size([20])
torch.Size([20, 20])


In [ ]:
@torch.no_grad()
def get_pred_probs(batch, model):
    graph_label = batch.y.clone().detach().cpu()
    graph_label = graph_label.squeeze(-1)
    batch_z = model.encode_first_stage(batch)
    node_decode, edge_decode, graph_decode = model.decode_first_stage(batch_z)
    graph_decode = graph_decode.squeeze(-1)
    probabilities = torch.sigmoid(graph_decode).cpu()

    return probabilities, batch_z, node_decode, edge_decode, graph_decode

@torch.no_grad()
def fidelity(model, decoded, loader):
    # all_orig_probs = []
    # all_cf_probs = []
    fid_drops = []
    for i, batch in loader:
        graph_label = batch.y.clone().detach().cpu()
        graph_label = graph_label.squeeze(-1).int()
        
        orig_probs, _, _, _, _ = get_pred_probs(batch, model) #bs
        orig_probs = torch.stack(1-orig_probs, orig_probs, dim=1) #bs x 2
        #all_orig_probs.append(orig_probs)
        
        graph_decode, node_decode, edge_decode = decoded[i]
        decoded_batch = batch
        print('decoded_batch', decoded_batch)
        decoded_batch.x = node_decode
        decoded_batch.edge_decode = edge_decode
        

        #either calculate with graph_decode which is kindddda cheating bc we dont construct graph and
        #embed back to alte
        cf_probs, _, _, _, _ = get_pred_probs(decoded_batch, model)
        orig_probs = torch.stack(1-orig_probs, orig_probs, dim=1) #bs x 2
        
        fid_drop = (1 - cf_probs.gather(1, labels).view(-1)).detach().cpu().numpy()
        fid_drop = np.mean(fid_drop)
        
        #all_labels.append(graph_label)

    all_probs = torch.cat(all_probs, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

@torch.no_grad()
def eval_cf(model, loader):
    

        
        